# Scrapping Politifact Dataset


Team: CloudBond


Fake news can take on many forms, including photoshopped images, fake user-generated content or spoofed accounts, network-based content designed to appeal to a particular organization or group, as well as knowledge-based stories that contain a scientific or reasonable explanation of unresolved issues, often resulting in the spread of false information.

The dataset used is scrapped data from Politifact. Along with scraping the news addtional source information for the news is also scrapped.


### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import csv
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import re
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import matplotlib.pyplot as plt
from scipy import sparse

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Import Posts data collected

In [ ]:
cd 'drive/MyDrive'

[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive


In [ ]:
df = pd.read_csv("NewsData.csv",sep=',')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18210 entries, 0 to 18209
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     18210 non-null  int64 
 1   Statement      18210 non-null  object
 2   Link           18210 non-null  object
 3   Date           18210 non-null  object
 4   Source         18210 non-null  object
 5   Context        18210 non-null  object
 6   ContextSource  12459 non-null  object
 7   Label          18210 non-null  object
 8   TRUE           18210 non-null  int64 
 9   MostlyTrue     18210 non-null  int64 
 10  HalfTrue       18210 non-null  int64 
 11  MostlyFalse    18210 non-null  int64 
 12  FALSE          18210 non-null  int64 
 13  PantOnFire     18210 non-null  int64 
dtypes: int64(7), object(7)
memory usage: 1.9+ MB


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18210 entries, 0 to 18209
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     18210 non-null  int64 
 1   Statement      18210 non-null  object
 2   Link           18210 non-null  object
 3   Date           18210 non-null  object
 4   Source         18210 non-null  object
 5   Context        18210 non-null  object
 6   ContextSource  12459 non-null  object
 7   Label          18210 non-null  object
 8   TRUE           18210 non-null  int64 
 9   MostlyTrue     18210 non-null  int64 
 10  HalfTrue       18210 non-null  int64 
 11  MostlyFalse    18210 non-null  int64 
 12  FALSE          18210 non-null  int64 
 13  PantOnFire     18210 non-null  int64 
dtypes: int64(7), object(7)
memory usage: 1.9+ MB


In [ ]:
df.head()

,Unnamed: 0,Statement,Link,Date,Source,Context,ContextSource,Label,TRUE,MostlyTrue,HalfTrue,MostlyFalse,FALSE,PantOnFire
0,0,Says Nancy Pelosi has “suspicious ties” to a v...,https://www.politifact.com/factchecks/2020/nov...,"ember 9, 2020",Clever Journeys,Bloggers,NaN,barely-true,1,3,3,6,25,59
1,1,“Mitt Romney has been asked to lead Health and...,https://www.politifact.com/factchecks/2020/nov...,"ember 9, 2020",The Buffalo Chronicle,Bloggers,NaN,FALSE,1,3,3,6,25,59
2,2,Says The Washington Times ran a front-page hea...,https://www.politifact.com/factchecks/2020/nov...,"ember 9, 2020",Facebook post,Viral image,NaN,FALSE,0,1,4,6,52,34
3,3,Says a New York Post cover used the headline “...,https://www.politifact.com/factchecks/2020/nov...,"ember 9, 2020",Facebook post,Viral image,NaN,FALSE,0,1,4,6,52,34
4,4,"Officials in Fairfax, Va., gave 100,000 Trump ...",https://www.politifact.com/factchecks/2020/nov...,"ember 7, 2020",Facebook post,Facebook posts,www.facebook.com/,barely-true,1,3,7,10,51,25


In [ ]:
df.shape

(18210, 14)

##Scrape Source Information for the Posts

###Scraping in Batches

In [ ]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

pagesToGet= 50
#pagesToGet= 2
upperframe=[]  
for page in range(0,pagesToGet):
    print('processing page :', page+1)
    link = df['Link'][page]
    url = str(link)+'#sources'
    print(url)
    
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        curr_page=requests.get(url)  
        #print(page.text)                           # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.
    time.sleep(2)   
    soup=BeautifulSoup(curr_page.text,'html.parser')
    frame=[]
    #links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    
    article_tag=soup.find('article',attrs={'class':'m-superbox__content'})
    article_title=soup.find('div',attrs={'class':'m-statement__quote'})
    #print(len(links))
    filename="News_Sources1.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Title, Author, Author_Link\n"
    f.write(headers)
    
    

    authors = article_tag.find_all("p")
    for author in authors:
      Title = article_title.text.strip()
      Author = author.text.strip()
      if(author.find('a')):
        Author_Link = author.find('a')['href'].strip()
      else:
        Author_Link = '0'
      frame.append((Title,Author,Author_Link))
      f.write(Title+","+Author+","+Author_Link+"\n")
    upperframe.extend(frame)
f.close()


processing page : 1
https://www.politifact.com/factchecks/2020/nov/09/blog-posting/trump-supporters-falsely-tie-nancy-pelosi-broader-/#sources
processing page : 2
https://www.politifact.com/factchecks/2020/nov/09/blog-posting/no-joe-biden-hasnt-asked-mitt-romney-be-his-health/#sources
processing page : 3
https://www.politifact.com/factchecks/2020/nov/09/viral-image/no-washington-times-didnt-run-president-gore-cover/#sources
processing page : 4
https://www.politifact.com/factchecks/2020/nov/09/viral-image/no-new-york-post-didnt-run-youre-fired-cover/#sources
processing page : 5
https://www.politifact.com/factchecks/2020/nov/07/facebook-posts/no-fairfax-officials-virginia-didnt-give-100000-tr/#sources
processing page : 6
https://www.politifact.com/factchecks/2020/nov/07/facebook-posts/no-naacp-didnt-warn-about-white-nationalist-groups/#sources
processing page : 7
https://www.politifact.com/factchecks/2020/nov/07/facebook-posts/georgia-election-worker-falsely-accused-discarding/#sources
p

In [ ]:
data1=pd.DataFrame(upperframe, columns=['Title','Author','Author_Link'])
data1.to_csv('News_Sources1.csv')
data1.head()

,Title,Author,Author_Link
0,Says Nancy Pelosi has “suspicious ties” to a v...,"Clever Journeys, Pelosi Has Suspicious Ties Wi...",https://cleverjourneys.com/2020/11/06/pelosi-h...
1,Says Nancy Pelosi has “suspicious ties” to a v...,"Red State, Election System Responsible for 'Gl...",https://redstate.com/jenvanlaar/2020/11/07/ele...
2,Says Nancy Pelosi has “suspicious ties” to a v...,"OpenSecrets.org, Dominion Voting Systems lobby...",https://www.opensecrets.org/federal-lobbying/c...
3,Says Nancy Pelosi has “suspicious ties” to a v...,"Michigan Department of State, False claims fro...","https://www.michigan.gov/som/0,4669,7-192-4779..."
4,Says Nancy Pelosi has “suspicious ties” to a v...,"Detroit Free Press, Antrim County figures prom...",https://www.freep.com/story/news/politics/elec...


In [ ]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

pagesToGet= 100
#pagesToGet= 2
upperframe=[]  
for page in range(50,pagesToGet):
    print('processing page :', page+1)
    link = df['Link'][page]
    url = str(link)+'#sources'
    print(url)
    
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        curr_page=requests.get(url)  
        #print(page.text)                           # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.
    time.sleep(2)   
    soup=BeautifulSoup(curr_page.text,'html.parser')
    frame=[]
    #links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    
    article_tag=soup.find('article',attrs={'class':'m-superbox__content'})
    article_title=soup.find('div',attrs={'class':'m-statement__quote'})
    #print(len(links))
    filename="News_Sources2.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Title, Author, Author_Link\n"
    f.write(headers)
    
    

    authors = article_tag.find_all("p")
    for author in authors:
      Title = article_title.text.strip()
      Author = author.text.strip()
      if(author.find('a')):
        Author_Link = author.find('a')['href'].strip()
      else:
        Author_Link = '0'
      frame.append((Title,Author,Author_Link))
      f.write(Title+","+Author+","+Author_Link+"\n")
    upperframe.extend(frame)
f.close()


processing page : 51
https://www.politifact.com/factchecks/2020/nov/04/viral-image/no-whitmer-didnt-say-she-wouldnt-certify-election-/#sources
processing page : 52
https://www.politifact.com/factchecks/2020/nov/04/tweets/no-wisconsin-doesnt-have-more-ballots-cast-registe/#sources
processing page : 53
https://www.politifact.com/factchecks/2020/nov/04/facebook-posts/voting-machines-dont-read-ballots-filled-sharpies-/#sources
processing page : 54
https://www.politifact.com/factchecks/2020/nov/04/sean-hannity/sean-hannity-falsely-suggests-ilhan-omar-encourage/#sources
processing page : 55
https://www.politifact.com/factchecks/2020/nov/04/facebook-posts/no-trump-hasnt-won-election-and-neither-has-biden/#sources
processing page : 56
https://www.politifact.com/factchecks/2020/nov/04/facebook-posts/fact-checking-misleading-photo-caption-about-phila/#sources
processing page : 57
https://www.politifact.com/factchecks/2020/nov/04/donald-trump/trump-falsely-and-prematurely-claims-2020-presiden/#so

In [ ]:
data2=pd.DataFrame(upperframe, columns=['Title','Author','Author_Link'])
data2.to_csv('News_Sources2.csv')
data2.head()

,Title,Author,Author_Link
0,"Says Michigan Gov. Gretchen Whitmer said, “I w...","Facebook post, Nov. 3, 2020",https://archive.is/XLkOQ
1,"Says Michigan Gov. Gretchen Whitmer said, “I w...","USA Today, Trump rally crowd chants ‘lock her ...",https://www.usatoday.com/story/news/politics/e...
2,"Says Michigan Gov. Gretchen Whitmer said, “I w...","Dana Nessel tweet, Oct. 17, 2020",https://twitter.com/dananessel/status/13176371...
3,"Says Michigan Gov. Gretchen Whitmer said, “I w...","Michigan Secretary of State, Get to know your ...","https://www.michigan.gov/sos/0,4670,7-127-1633..."
4,"Says Michigan Gov. Gretchen Whitmer said, “I w...","Email interview with Chelsea Lewis, deputy pre...",0


In [ ]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

pagesToGet= 150
#pagesToGet= 2
upperframe=[]  
for page in range(100,pagesToGet):
    print('processing page :', page+1)
    link = df['Link'][page]
    url = str(link)+'#sources'
    print(url)
    
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        curr_page=requests.get(url)  
        #print(page.text)                           # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.
    time.sleep(2)   
    soup=BeautifulSoup(curr_page.text,'html.parser')
    frame=[]
    #links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    
    article_tag=soup.find('article',attrs={'class':'m-superbox__content'})
    article_title=soup.find('div',attrs={'class':'m-statement__quote'})
    #print(len(links))
    filename="News_Sources3.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Title, Author, Author_Link\n"
    f.write(headers)
    
    

    authors = article_tag.find_all("p")
    for author in authors:
      Title = article_title.text.strip()
      Author = author.text.strip()
      if(author.find('a')):
        Author_Link = author.find('a')['href'].strip()
      else:
        Author_Link = '0'
      frame.append((Title,Author,Author_Link))
      f.write(Title+","+Author+","+Author_Link+"\n")
    upperframe.extend(frame)
f.close()

processing page : 101
https://www.politifact.com/factchecks/2020/oct/30/nicole-galloway/does-gov-parson-actually-oppose-protections-preexi/#sources
processing page : 102
https://www.politifact.com/factchecks/2020/oct/30/facebook-posts/false-post-inflates-number-rejected-mail-ballots-m/#sources
processing page : 103
https://www.politifact.com/factchecks/2020/oct/30/donald-trump-jr/donald-trump-jr-claims-covid-19-deaths-almost-noth/#sources
processing page : 104
https://www.politifact.com/factchecks/2020/oct/30/chris-jacobs/pre-existing-conditions-jacobs-claim-needs-context/#sources
processing page : 105
https://www.politifact.com/factchecks/2020/oct/30/donald-trump/donald-trump-says-gdp-growth-was-historic-beat-pre/#sources
processing page : 106
https://www.politifact.com/factchecks/2020/oct/30/joe-biden/does-donald-trump-think-15-hour-too-much-essential/#sources
processing page : 107
https://www.politifact.com/factchecks/2020/oct/30/viral-image/no-photo-doesnt-show-joe-and-hunter-biden

In [ ]:
data3=pd.DataFrame(upperframe, columns=['Title','Author','Author_Link'])
data3.to_csv('News_Sources3.csv')
data3.head()

,Title,Author,Author_Link
0,"""Mike Parson opposes protections for pre-exist...","KCUR 89.3, Galloway Takes On The Challenge Of ...",https://www.kcur.org/politics-elections-and-go...
1,"""Mike Parson opposes protections for pre-exist...","Nicole Galloway, ""Even Now"" Ad, Sept. 26, 2020",https://nicolegalloway.com/news/galloway-misso...
2,"""Mike Parson opposes protections for pre-exist...","Email Exchange with Kevin Donohoe, Galloway Ca...",0
3,"""Mike Parson opposes protections for pre-exist...","Interview with Kevin Donohoe, Galloway Campaig...",0
4,"""Mike Parson opposes protections for pre-exist...","U.S. House of Representatives, The Affordable ...",http://housedocs.house.gov/energycommerce/ppac...


In [ ]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

pagesToGet= 200
#pagesToGet= 2
upperframe=[]  
for page in range(150,pagesToGet):
    print('processing page :', page+1)
    link = df['Link'][page]
    url = str(link)+'#sources'
    print(url)
    
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        curr_page=requests.get(url)  
        #print(page.text)                           # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.
    time.sleep(2)   
    soup=BeautifulSoup(curr_page.text,'html.parser')
    frame=[]
    #links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    
    article_tag=soup.find('article',attrs={'class':'m-superbox__content'})
    article_title=soup.find('div',attrs={'class':'m-statement__quote'})
    #print(len(links))
    filename="News_Sources4.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Title, Author, Author_Link\n"
    f.write(headers)
    
    

    authors = article_tag.find_all("p")
    for author in authors:
      Title = article_title.text.strip()
      Author = author.text.strip()
      if(author.find('a')):
        Author_Link = author.find('a')['href'].strip()
      else:
        Author_Link = '0'
      frame.append((Title,Author,Author_Link))
      f.write(Title+","+Author+","+Author_Link+"\n")
    upperframe.extend(frame)
f.close()

processing page : 151
https://www.politifact.com/factchecks/2020/oct/27/facebook-posts/no-photo-paul-pelosi-jr-doesnt-show-him-abusing-gi/#sources
processing page : 152
https://www.politifact.com/factchecks/2020/oct/27/sri-preston-kulkarni/texas-house-race-kulkarni-distorts-nehls-take-mask/#sources
processing page : 153
https://www.politifact.com/factchecks/2020/oct/26/facebook-posts/no-evidence-hunter-biden-had-photos-coons-daughter/#sources
processing page : 154
https://www.politifact.com/factchecks/2020/oct/26/donald-trump/trump-wrongly-accuses-dc-mayor-calling-removal-was/#sources
processing page : 155
https://www.politifact.com/factchecks/2020/oct/26/scott-perry/pa-rep-scott-perry-airs-debunked-claim-wuhan-lab-c/#sources
processing page : 156
https://www.politifact.com/factchecks/2020/oct/26/joni-ernst/bipartisanship-rankings-show-joni-ernst-toward-top/#sources
processing page : 157
https://www.politifact.com/factchecks/2020/oct/26/viral-image/viral-image-migrant-cages-2019-not-ob

Now the sources for the posts is scraped.

Save the data in a data frame and add resutls to the CSV

In [ ]:
data4=pd.DataFrame(upperframe, columns=['Title','Author','Author_Link'])
data4.to_csv('News_Sources4.csv')
data4.head()

,Title,Author,Author_Link
0,Says photo from Hunter Biden's laptop shows Pa...,"Facebook post, Oct. 26, 2020",https://archive.is/QcmY5
1,Says photo from Hunter Biden's laptop shows Pa...,"Reverse-image search, Oct. 27, 2020",0
2,Says photo from Hunter Biden's laptop shows Pa...,"National File, EXCLUSIVE: FBI Hunts For Inform...",https://archive.fo/OBimo
3,Says photo from Hunter Biden's laptop shows Pa...,"PolitiFact, A look behind the NY Post headline...",https://www.politifact.com/article/2020/oct/15...
4,Says photo from Hunter Biden's laptop shows Pa...,"Lead Stories, Fact Check: Photo Does NOT Show ...",https://leadstories.com/hoax-alert/2020/10/fac...


In [ ]:
data = data1.append(data2).append(data3).append(data4)

In [ ]:
data.head()

,Title,Author,Author_Link
0,Says Nancy Pelosi has “suspicious ties” to a v...,"Clever Journeys, Pelosi Has Suspicious Ties Wi...",https://cleverjourneys.com/2020/11/06/pelosi-h...
1,Says Nancy Pelosi has “suspicious ties” to a v...,"Red State, Election System Responsible for 'Gl...",https://redstate.com/jenvanlaar/2020/11/07/ele...
2,Says Nancy Pelosi has “suspicious ties” to a v...,"OpenSecrets.org, Dominion Voting Systems lobby...",https://www.opensecrets.org/federal-lobbying/c...
3,Says Nancy Pelosi has “suspicious ties” to a v...,"Michigan Department of State, False claims fro...","https://www.michigan.gov/som/0,4669,7-192-4779..."
4,Says Nancy Pelosi has “suspicious ties” to a v...,"Detroit Free Press, Antrim County figures prom...",https://www.freep.com/story/news/politics/elec...


In [ ]:
data.shape

(2049, 3)

In [ ]:
data.to_csv('News_Sources_final.csv')

In [ ]:
#data = pd.read_csv('/content/drive/My Drive/Alternus_Vera/Veracity/News_Sources_final.csv',sep=',')

In [ ]:
data.head()

,Title,Author,Author_Link
0,Says Nancy Pelosi has “suspicious ties” to a v...,"Clever Journeys, Pelosi Has Suspicious Ties Wi...",https://cleverjourneys.com/2020/11/06/pelosi-h...
1,Says Nancy Pelosi has “suspicious ties” to a v...,"Red State, Election System Responsible for 'Gl...",https://redstate.com/jenvanlaar/2020/11/07/ele...
2,Says Nancy Pelosi has “suspicious ties” to a v...,"OpenSecrets.org, Dominion Voting Systems lobby...",https://www.opensecrets.org/federal-lobbying/c...
3,Says Nancy Pelosi has “suspicious ties” to a v...,"Michigan Department of State, False claims fro...","https://www.michigan.gov/som/0,4669,7-192-4779..."
4,Says Nancy Pelosi has “suspicious ties” to a v...,"Detroit Free Press, Antrim County figures prom...",https://www.freep.com/story/news/politics/elec...


In [ ]:
data.shape

(2049, 3)

#References

https://www.politifact.com

https://towardsdatascience.com/logistic-regression-a-simplified-approach-using-python-c4bc81a87c31


https://towardsdatascience.com/scraping-1000s-of-news-articles-using-10-simple-steps-d57636a49755

https://towardsdatascience.com/comparing-pandas-dataframes-to-one-another-c26853d7dda7

https://stackoverflow.com/questions/12497402/python-pandas-remove-duplicates-by-columns-a-keeping-the-row-with-the-highest

https://stackoverflow.com/questions/12497402/python-pandas-remove-duplicates-by-columns-a-keeping-the-row-with-the-highest


